In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pickle
with open('cropdata_all', 'rb') as f:
    data = pickle.load(f)
    
for key in data:
    print(key)
    
input_size = data['happy'].shape[1]
data_count = data['happy'].shape[0]
print("Data count = " + str(data_count))
emotion_count = len(data) - 1
print("Emotion count = " + str(emotion_count))
hidden_size = 100
batch_size = 20
MFCC_count = data['mfcc']

data.pop('mfcc', None)

happy
angry
neutral
surprise
mfcc
fear
sad
disgust
Data count = 399
Emotion count = 7


200

In [2]:
I = tf.placeholder(tf.float32, shape=(None, emotion_count, 1, input_size), name='I')
print(I.shape)

W = tf.Variable(tf.random_uniform((input_size, hidden_size)), name = 'W') / input_size
V = tf.Variable(tf.random_uniform((emotion_count, hidden_size, input_size)), name = 'V') / hidden_size
H = tf.einsum('ijkl,lm->ijkm', I, W)
Y = tf.concat([H[:,1:,:,:], H[:,0:1,:,:]], axis = 1)
print(Y.shape)
print(H.shape)
O = tf.einsum('ijkl,jlm->ijkm', H, V)
print(O.shape)

(?, 7, 1, 6600)
(?, 7, 1, 100)
(?, 7, 1, 100)
(?, 7, 1, 6600)


In [3]:
loss = tf.reduce_mean(tf.square(O - I)) + tf.reduce_mean(tf.square(H - Y))
# for i in range(batch_size):
# #     for j in range(emotion_count-2):
#     loss += tf.reduce_mean(tf.square(H[i][0] - H[i][1]))
#     loss += tf.reduce_mean(tf.square(H[i][1] - H[i][2]))
#     loss += tf.reduce_mean(tf.square(H[i][2] - H[i][0]))
# #     loss += tf.reduce_mean(tf.square(H[i][3] - H[i][4]))
# #     loss += tf.reduce_mean(tf.square(H[i][4] - H[i][5]))
# #     loss += tf.reduce_mean(tf.square(H[i][5] - H[i][6]))
# #     loss += tf.reduce_mean(tf.square(H[i][6] - H[i][0]))

    

In [4]:
train = tf.train.AdamOptimizer(0.1).minimize(loss)

In [5]:
#
# Preparing data.
l = list(data.keys())
l.sort()
print(l)
m = [data[key] for key in l]
# print(l.sort())
newdata = np.concatenate(m, axis = 1)
print(newdata.shape)
newdata = newdata.reshape(data_count, emotion_count, 1, input_size)


# newdata = newdata[:batch_size][:][:][:]
print(newdata.shape)

assert np.all(data['angry'][0] == newdata[0][0])

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
(399, 46200)
(399, 7, 1, 6600)


In [6]:
def createDataRange(start, end):
    return newdata[start:end][:][:][:]

In [23]:
saver = tf.train.Saver()

WW = None
VW = None

TRAINING_STEPS = 200
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) 
    
    # loading network from file.
    try:
        saver.restore(sess, "./modelall.ckpt")
        print("Loaded successfully.")
    except Exception as e:
        print('Cant.')
    
    # Actual training.
    for step in range(TRAINING_STEPS):
        for i in range(0, data_count, batch_size):
            loss_val = sess.run([train, loss], feed_dict={I:createDataRange(i, i+batch_size)})[1]
        if step % 10 == 0:
            print(str(step) + " " + str(loss_val))
    
    # Getting the actual values.
    WW = W.eval()
    VW = V.eval()
    print(tf.trainable_variables())
    
    # Saving the model file.
    save_path = saver.save(sess, './modelall.ckpt')
    print('Model saved in file: %s ' % save_path)

Loaded successfully.
0 24.7046
10 22.9668
20 23.601
30 23.6178
40 24.6869
50 26.2841
60 24.8944
70 24.3432
80 22.5317
90 22.6423
100 27.0021
110 22.7264
120 22.1771
130 22.197
140 22.5085
150 24.3507
160 24.0057
170 23.8133
180 22.4089
190 22.9694
[<tensorflow.python.ops.variables.Variable object at 0x000001F861B58240>, <tensorflow.python.ops.variables.Variable object at 0x000001F8620C3B38>]
Model saved in file: ./modelall.ckpt 


In [14]:
print(WW.shape)
print(VW.shape)

(6600, 100)
(7, 100, 6600)


In [15]:
#
#   Now, we recon the output vectors to listen to sound!
#

import librosa
# import numpy as np
from IPython.lib.display import Audio

def invlogamplitude(S):
    """librosa.logamplitude is actually 10_log10, so invert that."""
    return 10.0**(S/10.0)

def recon(mfccs, n_mfcc):
    
    sr = 22050
#     mfccs.reshape((1, input_size))
    mfccs = mfccs.reshape((n_mfcc, input_size // n_mfcc), order='F')
#     print(mfccs.shape)
    
#     n_mfcc = mfccs.shape[0]
    n_mel = 128
    dctm = librosa.filters.dct(n_mfcc, n_mel)
    n_fft = 2048
    mel_basis = librosa.filters.mel(sr, n_fft)

    bin_scaling = 1.0/np.maximum(0.0005, np.sum(np.dot(mel_basis.T, mel_basis),
    axis=0))

    recon_stft = bin_scaling[:, np.newaxis] * np.dot(mel_basis.T,
    invlogamplitude(np.dot(dctm.T, mfccs)))

    excitation = np.random.randn(mfccs.shape[1] * 510)
    E = librosa.stft(excitation)
    recon = librosa.istft(E/np.abs(E)*np.sqrt(recon_stft))

#     print(mfccs.shape)
    return Audio(recon, rate=sr)

In [27]:
# print(data['happy'][0].shape)
testinp = np.array([data['neutral'][2]])
print(testinp.shape)

recon(testinp, 200)

(1, 6600)


In [31]:
# ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
hh = np.matmul(testinp, WW)
print(hh.shape)

oo = np.matmul(hh, VW[1])
print(oo.shape)

print(np.max(oo))

recon(oo * 2 , 200)

(1, 100)
(1, 6600)
99.4921444023
